In [1]:
print("This is jupyter notebook running inside a virtual environment")

This is jupyter notebook running inside a virtual environment


In [2]:
import openai

In [3]:
pip show transformers

Name: transformers
Version: 4.41.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /Users/shreyashprashu/.pyenv/versions/3.8.10/envs/mypyvenvenv38/lib/python3.8/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers.utils import logging
logging.set_verbosity_error()

In [5]:
from transformers import pipeline

2024-06-10 23:33:51.929929: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
chatbot = pipeline(task="conversational",
                   model="facebook/blenderbot-400M-distill")

In [7]:
user_message = """
What are some fun activities I can do in Bangalore?
"""

In [8]:
from transformers import Conversation

In [9]:
conversation = Conversation(user_message)

In [10]:
print(conversation)

Conversation id: b1a86bd2-fea9-4ec1-827f-5cbc294fb674
user: 
What are some fun activities I can do in Bangalore?




In [11]:
conversation = chatbot(conversation)

In [12]:
print(conversation)

Conversation id: b1a86bd2-fea9-4ec1-827f-5cbc294fb674
user: 
What are some fun activities I can do in Bangalore?

assistant:  Well, Banglore is a country located in the southeastern region of South Asia.



In [14]:
conversation.add_message({"role":"user","content":""" what other cities are located nearby the city we are talking about? """})

In [15]:
conversation

Conversation id: b1a86bd2-fea9-4ec1-827f-5cbc294fb674
user: 
What are some fun activities I can do in Bangalore?

assistant:  Well, Banglore is a country located in the southeastern region of South Asia.
user:  what other cities are located nearby the city we are talking about? 

In [ ]:
conv = chatbot(conversation)

In [4]:
from huggingface_hub import login
login()

In [5]:
from transformers import pipeline

In [6]:
chatbot = pipeline("text-generation", model="meta-llama/Meta-Llama-3-8B")

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
def chat_with_llama3(user_input):
    # Initialize the conversation with the user's input
    conversation = [{"role": "user", "content": user_input}]

    # Generate a response from the chatbot
    response = chatbot(conversation, do_sample=False, max_new_tokens=50)

    # Extract the chatbot's response
    chatbot_response = response[0]["generated_text"][1]["content"]

    return chatbot_response

In [ ]:
# Test the chatbot
user_input = "What's the best way to travel to Europe?"
response = chat_with_llama3(user_input)
print(response)

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.
/Users/shreyashprashu/.pyenv/versions/3.8.10/envs/mypyvenvenv38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/shreyashprashu/.pyenv/versions/3.8.10/envs/mypyvenvenv38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set t

In [ ]:
# Follow-up question
user_input = "What about traveling by train?"
response = chat_with_llama3(user_input)
print(response)

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("\"")
]

input_ids = pipeline.tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
).to(pipeline.device)

attention_mask = input_ids.ne(pipeline.tokenizer.pad_token_id).to(pipeline.device)

outputs = pipeline.model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

response = outputs[0][input_ids.shape[-1]:]
print(pipeline.tokenizer.decode(response, skip_special_tokens=True))

In [ ]:
pip install accelerate